In [1]:
! pip install sodapy

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [24]:
# Import dependencies
import requests
import pandas as pd
from pprint import pprint
from sodapy import Socrata
import datetime

In [18]:
# Retrieve data using Socrata function provided by sodapy (API developers)
client = Socrata("data.cityofchicago.org", None)
results = client.get("6zsd-86xi", year=2018, limit=500000)

# Convert  results to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.head()

,arrest,beat,block,case_number,community_area,date,description,district,domestic,fbi_code,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,False,0524,117XX S SANGAMON ST,JB140548,53,2018-02-04T01:36:00.000,FIRST DEGREE MURDER,005,False,01A,...,41.68073915,"{'type': 'Point', 'coordinates': [-87.64566114...",STREET,-87.645661144,HOMICIDE,2018-02-11T15:57:41.000,34,1172100,1827003,2018
1,False,2433,011XX W GRANVILLE AVE,JB108189,77,2018-01-08T06:50:00.000,AUTOMOBILE,024,False,07,...,41.99456734,"{'type': 'Point', 'coordinates': [-87.65901631...",STREET,-87.659016317,MOTOR VEHICLE THEFT,2018-05-04T15:51:04.000,48,1167554,1941332,2018
2,False,2411,025XX W TOUHY AVE,JB108139,2,2018-01-07T21:00:00.000,AUTOMOBILE,024,False,07,...,42.01220081,"{'type': 'Point', 'coordinates': [-87.69390481...",STREET,-87.69390481,MOTOR VEHICLE THEFT,2018-05-04T15:51:04.000,50,1158022,1947685,2018
3,False,1834,004XX N MICHIGAN AVE,JB108319,8,2018-01-05T18:10:00.000,RETAIL THEFT,018,False,06,...,41.890384236,"{'type': 'Point', 'coordinates': [-87.62410991...",DEPARTMENT STORE,-87.624109917,THEFT,2018-05-04T15:51:04.000,42,1177357,1903444,2018
4,False,2223,097XX S HALSTED ST,JB108166,73,2018-01-08T04:09:00.000,ATTEMPT FORCIBLE ENTRY,022,False,05,...,41.71716022,"{'type': 'Point', 'coordinates': [-87.64309365...",RESTAURANT,-87.643093654,BURGLARY,2018-05-04T15:51:04.000,21,1172695,1840280,2018


In [29]:
# Extract useful rows into 'df' which will be saved to csv after munging
df = results_df[['case_number', 'date', 'year', 'description', 'primary_type', 
                 'fbi_code', 'latitude', 'longitude', 'domestic', 'arrest']]

# Parse datetime into date and time.
df[['date_parsed', 'time_parsed']] = df['date'].str.split('T', expand=True)
df = df.drop(columns=['date', 'year'])
df = df.rename(columns = {'date_parsed': 'date', 'time_parsed': 'time'})
df.head()

df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df['day'] = pd.DatetimeIndex(df['date']).day
df.head()

C:\Users\Laura Schmidt\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,case_number,description,primary_type,fbi_code,latitude,longitude,domestic,arrest,date,time,month,year,day
0,JB140548,FIRST DEGREE MURDER,HOMICIDE,01A,41.68073915,-87.645661144,False,False,2018-02-04,01:36:00.000,2,2018,4
1,JB108189,AUTOMOBILE,MOTOR VEHICLE THEFT,07,41.99456734,-87.659016317,False,False,2018-01-08,06:50:00.000,1,2018,8
2,JB108139,AUTOMOBILE,MOTOR VEHICLE THEFT,07,42.01220081,-87.69390481,False,False,2018-01-07,21:00:00.000,1,2018,7
3,JB108319,RETAIL THEFT,THEFT,06,41.890384236,-87.624109917,False,False,2018-01-05,18:10:00.000,1,2018,5
4,JB108166,ATTEMPT FORCIBLE ENTRY,BURGLARY,05,41.71716022,-87.643093654,False,False,2018-01-08,04:09:00.000,1,2018,8


In [30]:
#Save output to csv
df.to_csv('2018_crime.csv', index=False)